# Hamming Distances for Data-Set

In [ ]:
using QAOA, SparseArrays
using DataFrames, Arrow, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

using PyCall
np = pyimport("numpy")

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)
Base.show(io::IO, ::MIME"text/latex", df::AbstractDataFrame) = show(df, backend=:html, show_row_number=true, highlighters=:tf_html_default)

In [ ]:
N = 9
N = 11
N = 13
N = 15
N = 17
N = 19

patterns_dict = Dict(
    9  => r"random_SK_instance_N_9_seed_(\d+)\.h5",
    11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
    13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
    15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
    17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
    19 => r"random_SK_instance_N_19_seed_(\d+)\.h5"
);

## Loading

In [ ]:
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> occursin("random_SK", x), instance_names)
filter!(x -> !occursin("results", x), instance_names);

In [ ]:
ordered_seeds = []
all_mean_field_energies = Dict()
all_mean_field_sols = Dict()
all_eigenvals = Dict()
all_eigenstates = Dict()

for (k, instance_name) in enumerate(instance_names[1:end])
    seed = match(patterns_dict[N], instance_name)[1]    
    print(k, ", ")

    # eigenvalues and -vectors
    λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    all_eigvecs = h5read(folder_name * instance_name, "exact_ARPACK_LM_lowest_eigvecs")

    J_mat = h5read(folder_name * instance_name, "J")
    mf_problem = Problem(0, J_mat)

    # # mean-field solutions
    T_final = 32768.
    tol = 1e-6
    mf_sol = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
    sigma_star = sign.(mf_sol)

    h = mf_problem.local_fields
    J = mf_problem.couplings
    E_star = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1]) 
    
    # continue if mean-fields finds optimal solution
    if isapprox(E_star, λ[1, end], atol=1e-5)
        continue
    end

    # fluctuations
    T_final = 32000
    tol = 1e-8
    npts = 2048
    all_flucs = h5read(folder_name * "results_" * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
    lyapunov_exponent = sum(all_flucs, dims=1)
    if sum(lyapunov_exponent) |> abs >= 1e4 # discard non-converged ones
        continue
    end    
        
    all_mean_field_energies[seed] = E_star
    all_mean_field_sols[seed] = sigma_star
    
    all_eigenstates[seed] = [all_eigvecs[end, :, n] for n in 1:1]
    all_eigenvals[seed] = [λ[n, :] for n in 1:1]
    push!(ordered_seeds, seed)
end

In [ ]:
ordered_seeds |> size

In [ ]:
σ̂_0 = sparse([1. 0im; 0 1])
σ̂_x = sparse([0im 1.; 1 0])
σ̂_y = sparse([0. -1im; 1im 0])
σ̂_z = sparse([1. 0im; 0 -1]);

Zs = []
for i in 1:N-1
    idz = [σ̂_0 for _ in 1:N - 1]
    idz[i] = σ̂_z
    push!(Zs, reduce(kron, idz))
end

In [ ]:
avg_Delta_E = 0.
avg_hamming = 0.
for seed in ordered_seeds
    gs = all_eigenstates[seed][1]
    hamming = sum(abs.([gs' * Zs[i] * gs |> real for i in 1:N-1] .- all_mean_field_sols[seed])) / 2
    avg_Delta_E += abs(all_mean_field_energies[seed] - all_eigenvals[seed][1][end])
    avg_hamming += hamming

    # println(all_mean_field_sols[seed])
    # println([gs' * Zs[i] * gs |> real for i in 1:N-1])
    # println(hamming, ", ", all_mean_field_energies[seed], ", ", all_eigenvals[seed][1][end])
end
    
avg_Delta_E = avg_Delta_E / length(ordered_seeds)
avg_hamming = avg_hamming / length(ordered_seeds)

avg_Delta_E |> println
avg_hamming |> println

$N = 8$: $\bar{\Delta E} = 0.0979$, $\bar{d}_{\mathrm{Hamming}} = 5.9825$

$N = 10$: $\bar{\Delta E} = 0.1245$, $\bar{d}_{\mathrm{Hamming}} = 7.0217$

$N = 12$: $\bar{\Delta E} = 0.1589$, $\bar{d}_{\mathrm{Hamming}} = 7.8736$

$N = 14$: $\bar{\Delta E} = 0.3300$, $\bar{d}_{\mathrm{Hamming}} = 9.1162$

$N = 16$: $\bar{\Delta E} = 0.2744$, $\bar{d}_{\mathrm{Hamming}} = 10.0000$

$N = 18$: $\bar{\Delta E} = 0.2539$, $\bar{d}_{\mathrm{Hamming}} = 11.0364$

In [ ]:
avg_Delta_Es = [0.0979, 0.1245, 0.1589, 0.3300, 0.2744, 0.2539]
avg_hammings = [5.9825, 7.0217, 7.8736, 9.1162, 10.0000, 11.0364];

In [ ]:
avg_Delta_Es ./ [8, 10, 12, 14, 16, 18] |> println
avg_hammings ./ [8, 10, 12, 14, 16, 18] |> println